In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from tools.standard_name_utils import optimize_parameters
from tools.common_utils import highlight_diff, read_jsonl

# model_path = "../ChatGLM2-6B/ptuning/output/checkpoint-100"
model_path = "../../train-result/2024-06-17/checkpoint-500"
top_p=0.7 
temperature=0.95

/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()
model = model.eval()

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
Loading checkpoint shards: 100%|██████████| 6/6 [01:46<00:00, 17.80s/it]


In [3]:
response, history = model.chat(tokenizer, "你好", history=[], top_p=top_p, temperature=temperature)
print(response)

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


你好👋！我是人工智能助手 ChatGLM3-6B，很高兴见到你，欢迎问我任何问题。


### 评估一下训练数据的情况

In [ ]:
json_file_path = 'data/train.json'
data_list = read_jsonl(json_file_path)

df = pd.DataFrame(data_list)
df.head()

In [ ]:
from random import random

total_eval_count = 0.0
correct_count = 0.0

for index, row in df.iterrows():
    if random() <= 0.9:
        continue
    
    input = row["input"]
    output = row["output"]
    response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
    
    total_eval_count += 1
    if response == output:
        correct_count += 1
    else:
        print("-----data index-----")
        print(index)
        print("-----query input-----")
        print(input)
        print("-----output diff-----")
        print(highlight_diff(output, response))
        print(response)
        print()
    
    if total_eval_count == 100:
        break
    
print("预测正确的比例：" + f"{correct_count / total_eval_count :.2%}")

In [4]:
json_file_path = 'data/dev.json'
data_list = read_jsonl(json_file_path)

df = pd.DataFrame(data_list)
df.head()

,input,output
0,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
1,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
2,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
3,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
4,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."


In [5]:
data_stock = pd.read_excel('data/标准名.xlsx',sheet_name='股票标准名')
data_fund = pd.read_excel('data/标准名.xlsx',sheet_name='基金标准名')

fund_standard_name = data_fund['标准基金名称'].to_list()
stock_standard_name = data_stock['标准股票名称'].to_list()

In [ ]:
stock_standard_name[:5]

In [7]:
from random import random

total_eval_count = 0.0
correct_count = 0.0

for index, row in df.iterrows():
    if random() <= 0.9:
        continue
    
    input = row["input"]
    output = row["output"]
    response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
    
    optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)
    
    total_eval_count += 1
    if optimized_resp == output:
        correct_count += 1
    else:
        print("-----data index-----")
        print(index)
        print("-----query input-----")
        print(input)
        print("-----output diff-----")
        print(highlight_diff(output, optimized_resp))
        print(optimized_resp)
        print()
    
    if total_eval_count == 20:
        break
    
print("预测正确的比例：" + f"{correct_count / total_eval_count :.2%}")

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-----data index-----
5
-----query input-----
你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：昨天成交量是70万8300股，而上个月最低价是5.71元的股票有哪些？ 
 query中提到的产品标准名可能是：兴全中证800六个月持有期指数增强型证券投资基金C类、兴全中证800六个月持有期指数增强型证券投资基金A类、富国医药成长30股票型证券投资基金、申万菱信沪深300价值指数证券投资基金C类、申万菱信沪深300价值指数证券投资基金A类、大成睿裕六个月持有期股票型证券投资基金C类、大成睿裕六个月持有期股票型证券投资基金A类、富国沪深300基本面精选股票型证券投资基金C类、富国沪深300基本面精选股票型证券投资基金A类、招商科技动力3个月滚动持有股票型证券投资基金C类、招商科技动力3个月滚动持有股票型证券投资基金A类、安信均衡成长18个月持有期混合型证券投资基金C类、安信均衡成长18个月持有期混合型证券投资基金A类、中航量化阿尔法六个月持有期股票型证券投资基金C类、中航量化阿尔法六个月持有期股票型证券投资基金A类、中庚港股通价值18个月封闭运作股票型证券投资基金、申万菱信沪深300价值交易型开放式指数证券投资基金、大成尊享18个月持有期混合型发起式证券投资基金C类、大成尊享18个月持有期混合型发起式证券投资基金A类、国投瑞银港股通6个月定期开放股票型发起式证券投资基金、浙商汇金卓越优选3个月持有期股票型基金中基金(FOF)、新华精选成长主题3个月持有期混合型基金中基金(FOF)、同泰积极配置3个月持有期股票型基金中基金(FOF)C类、同泰积极配置3个月持有期股票型基金中基金(FOF)A类、大成沪深300指数证券投资基金、嘉实低价策略股票型证券投资基金、申万宏源季季优选3个月滚动持有债券型集合资产管理计划C类、申万宏源季季优选3个月滚动持有债券型集合资产管理计划A类、工银瑞信中证500六个月持有期指数增强型证券投资基金C类、工银瑞信中证500六个月持有期指数增强型证券投资基金A类、天弘睿享3个月持有期混合型发起式基金中基金(FOF)C类、天弘睿享3个月持有期混合型发起式基金中基金(FOF)A类、银华中国梦30股票型证券投资基金、南方天元新产业股票型证券投资基金、长江量化科技

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-----data index-----
7
-----query input-----
你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：我5000块要全投鹏华一带一路(LOF)A，扣除认购费用后真正可以投入的金额是多少 
 query中提到的产品标准名可能是：鹏华中证一带一路主题指数型证券投资基金(LOF)A类、鹏华中证一带一路主题指数型证券投资基金(LOF)C类、鹏华中证500指数证券投资基金(LOF)A类、易方达标普500指数证券投资基金(LOF)A类、富国中证500指数增强型证券投资基金(LOF)A类、中信保诚中证500指数型证券投资基金(LOF)A类、富国中证1000指数增强型证券投资基金(LOF)A类、财通资管中证500指数增强型证券投资基金(LOF)A类、西部利得中证500指数增强型证券投资基金(LOF)A类、汇添富中证500指数型发起式证券投资基金(LOF)A类、易方达标普500指数证券投资基金(LOF)(美元)A类、鹏华中证500指数证券投资基金(LOF)C类、易方达上证50指数证券投资基金(LOF)A类、广发深证100指数证券投资基金(LOF)A类、海富通中证100指数证券投资基金(LOF)A类、易方达标普500指数证券投资基金(LOF)C类、富国中证500指数增强型证券投资基金(LOF)、宏利中证500指数增强型证券投资基金(LOF)、融通巨潮100指数证券投资基金(LOF)A/B类、中金中证优选300指数证券投资基金(LOF)A类、广发中证500交易型开放式指数证券投资基金联接基金(LOF)A类、南方中证500交易型开放式指数证券投资基金联接基金(LOF)A类、鹏华中证空天一体军工指数证券投资基金(LOF)A类、长盛中证申万一带一路主题指数证券投资基金(LOF)、浙商沪深300指数增强型证券投资基金(LOF)A类、易方达中小企业100指数证券投资基金(LOF)A类、建信沪深300指数增强型证券投资基金(LOF)A类、富国中证500指数增强型证券投资基金(LOF)C类、大成中华沪深港300指数证券投资基金(LOF)A类、兴全沪深300指数增强型证券投资基金(LOF)A类、中信保诚沪深300指数型证券投资基金(LOF)A类、中信保诚中证500指数型证券投资基金(LOF)C类、银华中证中票5

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

-----data index-----
175
-----query input-----
你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：我想知道如果我昨天买大化的股票，我今天卖出的话对比昨天的买入价赚了多少？ 
 query中提到的产品标准名可能是：九泰天利量化股票型证券投资基金C类、九泰天利量化股票型证券投资基金A类、九泰天辰量化新动力股票型证券投资基金、天弘文化新兴产业股票型证券投资基金C类、天弘文化新兴产业股票型证券投资基金A类、光大保德信创业板量化优选股票型证券投资基金C类、光大保德信创业板量化优选股票型证券投资基金A类、创金合信上证超大盘量化精选股票型发起式证券投资基金C类、创金合信上证超大盘量化精选股票型发起式证券投资基金A类、大成优选股票型证券投资基金、大成睿鑫股票型证券投资基金C类、大成睿鑫股票型证券投资基金A类、嘉实量化精选股票型证券投资基金、南方量化成长股票型证券投资基金、九泰久信量化股票型证券投资基金、长信量化中小盘股票型证券投资基金、建信多因子量化股票型证券投资基金、国泰大农业股票型证券投资基金C类、国泰大农业股票型证券投资基金A类、国泰大健康股票型证券投资基金C类、国泰大健康股票型证券投资基金A类、南方天元新产业股票型证券投资基金、中银大健康股票型证券投资基金C类、中银大健康股票型证券投资基金A类、鹏华文化传媒娱乐股票型证券投资基金、长城量化精选股票型证券投资基金C类、长城量化精选股票型证券投资基金A类、长城量化小盘股票型证券投资基金C类、长城量化小盘股票型证券投资基金A类、英大国企改革主题股票型证券投资基金、泰达宏利量化增强股票型证券投资基金、汇丰晋信大盘股票型证券投资基金C类、汇丰晋信大盘股票型证券投资基金A类、景顺长城量化精选股票型证券投资基金、景顺长城量化小盘股票型证券投资基金、摩根大盘蓝筹股票型证券投资基金C类、摩根大盘蓝筹股票型证券投资基金A类、建信量化事件驱动股票型证券投资基金、大成科技消费股票型证券投资基金C类、大成科技消费股票型证券投资基金A类、大成消费精选股票型证券投资基金C类、大成消费精选股票型证券投资基金A类、大成品质医疗股票型证券投资基金C类、大成品质医疗股票型证券投资基金A类、大成医药健康股票型证券投资基金C类、大成医药健康股票型证券投资基金A类、博

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-----data index-----
202
-----query input-----
你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：想了解下南方恒生A基金由成立至今的年化收益率是怎样的，以及其在同类型基金中的排名如何 
 query中提到的产品标准名可能是：东方强化收益债券型证券投资基金、南方恒生交易型开放式指数证券投资基金联接基金C类、南方恒生交易型开放式指数证券投资基金联接基金A类、大成强化收益定期开放债券型证券投资基金、大成强化收益定期开放债券型证券投资基金、南方亚洲美元收益债券型证券投资基金C类、南方亚洲美元收益债券型证券投资基金A类、南方恒生中国企业交易型开放式指数证券投资基金联接基金C类、南方恒生中国企业交易型开放式指数证券投资基金联接基金A类、东方成长收益灵活配置混合型证券投资基金C类、东方成长收益灵活配置混合型证券投资基金A类、南方收益宝货币市场基金B类、南方收益宝货币市场基金A类、汇添富恒生指数分级证券投资基金之稳健收益类份额、汇添富恒生指数分级证券投资基金之积极收益类份额、南方绝对收益策略定期开放混合型发起式证券投资基金、银河强化收益债券型证券投资基金、诺安优化收益债券型证券投资基金、南方宝恒混合型证券投资基金C类、南方宝恒混合型证券投资基金A类、南方亚洲美元收益债券型证券投资基金(美元现汇)C类、南方亚洲美元收益债券型证券投资基金(美元现汇)A类、易方达保证金收益货币市场基金B类、易方达保证金收益货币市场基金A类、银华恒生中国企业指数分级证券投资基金之稳健收益类份额、银华恒生中国企业指数分级证券投资基金之积极收益类份额、南方安享绝对收益策略定期开放混合型发起式证券投资基金、南方卓享绝对收益策略定期开放混合型发起式证券投资基金、金鹰民富收益混合型证券投资基金C类、金鹰民富收益混合型证券投资基金A类、富国天丰强化收益债券型证券投资基金、华安强化收益债券型证券投资基金B类、华安强化收益债券型证券投资基金A类、民生加银新收益债券型证券投资基金C类、民生加银新收益债券型证券投资基金A类、易方达稳健收益债券型证券投资基金B类、易方达稳健收益债券型证券投资基金A类、南方浩达稳健优选一年持有期混合型基金中基金(FOF)C类、南方浩达稳健优选一年持有期混合型基金中基金(FOF)A类、南方浩盈进取

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


预测正确的比例：40.00%


In [ ]:
test_input = df.loc[35]["input"]
test_input

In [ ]:
test_input = df.loc[19]["input"]
test_input += " 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价"
response, history = model.chat(tokenizer=tokenizer, query=test_input, history=[], top_p=top_p, temperature=temperature)
optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)

output = df.loc[19]["output"]
print(highlight_diff(output, optimized_resp))
print(optimized_resp)

### 输出新的提交文件

In [ ]:
test_json_file_path = 'data/test_a.json'
test_data_list = read_jsonl(test_json_file_path)

test_df = pd.DataFrame(test_data_list)
test_df.head()

In [ ]:
total_eval_count = 0

with open('data/submit.txt','w', encoding="utf-8") as n:
    for index, row in test_df.iterrows():    
        input = row["input"]
        response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
        
        optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)
        
        total_eval_count += 1
        n.write(optimized_resp+'\n')
        if total_eval_count % 10 == 0:
            print("现在是第" + f"{total_eval_count}" + "条数据")